Start ganache-cli Ethereum emilator via

`npx ganache-cli -m "dawn finish orchard pluck festival genuine absorb van bike mirror kiss loop"`

Or run both notebook and node via `start.sh` script

In [36]:
from web3 import Web3
mnemonic = 'dawn finish orchard pluck festival genuine absorb van bike mirror kiss loop'
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.eth.defaultAccount = w3.eth.accounts[0]

Check if web3 up and running

In [37]:
w3.eth.getBlock('latest')

AttributeDict({'number': 0,
 'hash': HexBytes('0xbeb59fbecede65a6ee3a1af152c8bfd56e5f9c5137482a15814cbc7c126d9472'),
 'parentHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'mixHash': HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
 'nonce': HexBytes('0x0000000000000000'),
 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'),
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'transactionsRoo

Create Solidity smart contract.

You can test it in online Solidity IDE Remix

In [96]:
contract_source_code = b"""

pragma solidity ^0.5.7;

contract Auth {      
    function verify(address p, bytes32 hash, uint8 v, bytes32 r, bytes32 s) pure public returns(bool) {
        return ecrecover(hash, v, r, s) == p;
    }
}

"""

Compile the contract with solc Solidity compiler

In [97]:
import json
import subprocess
solc = subprocess.Popen(['solc', '--combined-json', 'bin,abi', '-'], stdout=subprocess.PIPE, stdin=subprocess.PIPE)
solc_output = solc.communicate(contract_source_code)

See the contract bytecode

In [98]:
bytecode = json.loads(solc_output[0])['contracts']['<stdin>:Auth']['bin']
bytecode

'608060405234801561001057600080fd5b50610185806100206000396000f3fe608060405234801561001057600080fd5b506004361061002b5760003560e01c8063506535f314610030575b600080fd5b61009d600480360360a081101561004657600080fd5b81019080803573ffffffffffffffffffffffffffffffffffffffff16906020019092919080359060200190929190803560ff16906020019092919080359060200190929190803590602001909291905050506100b7565b604051808215151515815260200191505060405180910390f35b60008573ffffffffffffffffffffffffffffffffffffffff1660018686868660405160008152602001604052604051808581526020018460ff1660ff1681526020018381526020018281526020019450505050506020604051602081039080840390855afa15801561012d573d6000803e3d6000fd5b5050506020604051035173ffffffffffffffffffffffffffffffffffffffff161490509594505050505056fea165627a7a7230582093a5ddcab62c2e993758d6e42615e75567392c49cb4606fd6ae140e1496c640b0029'

See the contract ABI interfaces

In [99]:
abi = json.loads(solc_output[0])['contracts']['<stdin>:Auth']['abi']
abi

'[{"constant":true,"inputs":[{"name":"p","type":"address"},{"name":"hash","type":"bytes32"},{"name":"v","type":"uint8"},{"name":"r","type":"bytes32"},{"name":"s","type":"bytes32"}],"name":"verify","outputs":[{"name":"","type":"bool"}],"payable":false,"stateMutability":"pure","type":"function"}]'

Instantiate contract fabric and deploy the contract on the net

In [100]:
AuthContract = w3.eth.contract(abi=abi, bytecode=bytecode)
tx_hash = AuthContract.constructor().transact()
tx_hash

HexBytes('0x5d41441f1ff689845c846e9b7c53dee91afa66f94ebbaef339d4dbf29289076e')

Get Tx receipt (see contractAddress)

In [101]:
tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
tx_receipt

AttributeDict({'transactionHash': HexBytes('0x5d41441f1ff689845c846e9b7c53dee91afa66f94ebbaef339d4dbf29289076e'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0xd571aab2c30df3d8f3430e9f0244536bf24f244d5ad208631d36a760a574212c'),
 'blockNumber': 7,
 'from': '0x90c08087af274b77516df05952273008fea2c4b9',
 'to': None,
 'gasUsed': 158277,
 'cumulativeGasUsed': 158277,
 'contractAddress': '0xAE2EB54D488e739d9044565Ed53846399879637B',
 'logs': [],
 'status': 1,
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

Initialize contract instance at given address

In [103]:
auth_contract = w3.eth.contract(
    address=tx_receipt.contractAddress,
    abi=abi,
)
auth_contract

Call the contract methods

In [105]:
auth_contract.functions.verify(
    '0x7E5F4552091A69125d5DfCb7b8C2659029395Bdf',
    bytes(0),
    1,
    bytes(b'\xff'*32),
    bytes(b'\xff'*32)).call()

False